# Library Calls

In [1]:
import pandas as pd
import numpy as np
import feather
import matplotlib.pyplot as plt

# Random State
np.random.seed(5590)

# Data Import

In [2]:
note_events = pd.read_csv('./data/NOTEEVENTS.csv.gz', compression='gzip', low_memory=False)

admissions_readmit = feather.read_dataframe('./data/export_for_notes.feather')

# Processing

Quick check for admissions_readmit dataframe.

In [3]:
admissions_readmit.head().T

,0,1,2,3,4
hadm_id,118464,173927,159656,158366,113808
readmit_30,0,0,0,0,0


In [4]:
admissions_readmit.shape

(45307, 2)

## Inspection / Basic Cleaning

In [5]:
note_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [6]:
note_events.shape

(2083180, 11)

In [7]:
note_events.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT'],
      dtype='object')

I prefer having a consistent lower case column name nomenclature.

In [8]:
note_events.columns = note_events.columns.str.lower()

In [9]:
note_events.columns

Index(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text'],
      dtype='object')

We need the hadm_id, iserror, category, description, and text columns for this analysis.

In [10]:
# Drop columns
drop_columns = [
    'row_id',
    'subject_id',
    'chartdate',
    'charttime',
    'storetime',
    'cgid',
    #'iserror'
]

note_events.drop(columns = drop_columns, inplace = True)

There are 866 out of over 2 million records that are in error. These records will be dropped.

In [11]:
print(note_events.iserror.value_counts())

1.0    886
Name: iserror, dtype: int64


In [12]:
note_events = note_events.query('iserror != 1')

In [13]:
note_events.shape

(2082294, 5)

With all values of iserror dropped we can drop the column.

In [14]:
note_events.drop(columns = ['iserror'], inplace = True)

In [15]:
note_events.shape

(2082294, 4)

After dropping unnecessary columns we will inspect the data frame to verify the required columns are there. Additionally we will check the data frame shape and check for missing values.

In [16]:
# Check Columns
note_events.head().T

,0,1,2,3,4
hadm_id,167853,107527,167118,196489,135453
category,Discharge summary,Discharge summary,Discharge summary,Discharge summary,Discharge summary
description,Report,Report,Report,Report,Report
text,Admission Date: [**2151-7-16**] Dischar...,Admission Date: [**2118-6-2**] Discharg...,Admission Date: [**2119-5-4**] D...,Admission Date: [**2124-7-21**] ...,Admission Date: [**2162-3-3**] D...


In [17]:
# NAs
note_events.isna().sum()

hadm_id        231829
category            0
description         0
text                0
dtype: int64

Per the MIMIC website if hadm_id is NA that means the patient was an outpatient. We are not interested in outpatient conditions for this analysis so rows with an NA can be dropped.

In [18]:
note_events.dropna(inplace = True)

In [19]:
# Check Columns
note_events.head().T

,0,1,2,3,4
hadm_id,167853,107527,167118,196489,135453
category,Discharge summary,Discharge summary,Discharge summary,Discharge summary,Discharge summary
description,Report,Report,Report,Report,Report
text,Admission Date: [**2151-7-16**] Dischar...,Admission Date: [**2118-6-2**] Discharg...,Admission Date: [**2119-5-4**] D...,Admission Date: [**2124-7-21**] ...,Admission Date: [**2162-3-3**] D...


In [20]:
# NAs
note_events.isna().sum()

hadm_id        0
category       0
description    0
text           0
dtype: int64

The column hadm_id was imported as a numeric value. It needs to be converted to a string to funciton correctly as an id.

In [21]:
note_events.hadm_id = note_events.hadm_id.astype('int64').astype(str)

In [22]:
print(note_events.category.value_counts())

Nursing/other        821258
Radiology            378920
Nursing              220384
Physician            139763
ECG                  138190
Discharge summary     59652
Echo                  34037
Respiratory           31629
Nutrition              9361
General                8144
Rehab Services         5386
Social Work            2603
Case Management         940
Pharmacy                100
Consult                  98
Name: category, dtype: int64


In [23]:
#print(note_events.description.value_counts())

## Grouping Text Notes

The goal is to create two data frames of notes. 

* Data Frame 1: All Notes in the original table, grouped by hadm_id, concatenated together
* Data Frame 2: Only Discharge Summary Notes (original and addendums), grouped by hadm_id, concatenated together

### Data Frame: All Notes

In [24]:
note_events_all = note_events.groupby(['hadm_id'])['text'].apply(lambda x: ' '.join(x)).reset_index()

### Data Frame 2: Discharge Summary

In [25]:
# Filter for only Discharge Summary Records
note_events_ds = note_events.query('category == "Discharge summary"')

There should only be Reports and Addendums in the Discharge Summary dataframe. The following code is to verify that.

In [26]:
print(
    note_events_ds.description.value_counts()
)

Report      55177
Addendum     4475
Name: description, dtype: int64


In [27]:
note_events_ds = note_events_ds.groupby(['hadm_id'])['text'].apply(lambda x: ' '.join(x)).reset_index()

**The different size of the two data frames implies that there are some records where there was no discharge report. Before going further import the final admissions hadm_id set and filter out already dropped records. This may close this gap.**

## Joining Readmission Data Frame 

In [28]:
admissions_readmit.readmit_30 = admissions_readmit.readmit_30.astype('int')

### All Notes Data Frame

In [29]:
note_events_all_merge = pd.merge(note_events_all,
                         admissions_readmit, 
                         left_on='hadm_id',
                         right_on='hadm_id',
                         how='left')

We need to check for NAs. The note_events data frame should have observations that have been dropped already from the admissions_readmit dataframe.

In [30]:
# NAs
note_events_all_merge.isna().sum()

hadm_id           0
text              0
readmit_30    13237
dtype: int64

We then drop these missing records.

In [31]:
note_events_all_merge.dropna(inplace = True)

In [32]:
note_events_all_merge.shape

(45124, 3)

### Discharge Summary Data Frame

In [33]:
note_events_ds_merge = pd.merge(note_events_ds,
                         admissions_readmit, 
                         left_on='hadm_id',
                         right_on='hadm_id',
                         how='left')

We need to check for NAs. The note_events data frame should have observations that have been dropped already from the admissions_readmit dataframe.

In [34]:
# NAs
note_events_ds_merge.isna().sum()

hadm_id          0
text             0
readmit_30    8851
dtype: int64

We then drop these missing records.

In [35]:
note_events_ds_merge.dropna(inplace = True)

In [36]:
note_events_ds_merge.shape

(43875, 3)

# Export

In [41]:
#All Notes
path = './data/text_processed_all.csv.gz'
note_events_all_merge.to_csv(path, compression='gzip', index=False)

# Discharge Summary Only
path = './data/text_processed_discharge_summary.csv.gz'
note_events_ds_merge.to_csv(path, compression='gzip', index=False)